In [1]:
import numpy as np
import os
from skimage.io import imread, imsave
from skimage.draw import rectangle

### Path to folders image, labels and output result files


In [8]:
image_path = 'D:\IT\Python\Ship_Detection\Airbus_Ship_Detection_Kaggle\Resource\Train\images\\'
label_path = 'D:\IT\Python\Ship_Detection\Airbus_Ship_Detection_Kaggle\Resource\Train\labels\\'
output_path = 'D:\IT\Python\Ship_Detection\Airbus_Ship_Detection_Kaggle\Resource\Train\outputs\\'

### Create mask function

In [6]:
def create_mask(image_shape, labels):
    #Create mask of zero
    mask = np.zeros(image_shape[:2], dtype = np.uint8)

    height, width = image_shape[:2]
    for label in labels:
        x_center, y_center, width_norm, heighr_norm = map(float, label.split())

        box_width = int(width_norm * width)
        box_height = int(heighr_norm * height)

        x_center_abs = int(x_center * width)
        y_center_abs = int(y_center * height)

        top_left_x = max(0, int(x_center_abs - box_width / 2))
        top_left_y = max(0, int(y_center_abs - box_height / 2))
        bottom_right_x = min(width, int(x_center_abs + box_width / 2))
        bottom_right_y = min(height, int(y_center_abs + box_height / 2))

        rr, cc = rectangle(start=(top_left_y, top_left_x), extent=(bottom_right_y - top_left_y, bottom_right_x - top_left_x), shape=image_shape[:2])
        mask[rr,cc] = 1 # Set the pixels in the rectangle to 1

    return mask

### Function check file in folders and return count files that match

In [9]:
def count_matching_filenames(image_folder, label_folder):
    # Get a list of filenames without extensions for each folder
    images_file = {os.path.splitext(file)[0] for file in os.listdir(image_path)}
    label_file = {os.path.splitext(file)[0] for file in os.listdir(label_path)}

    # Finding the intersection between two sets
    matching_files = images_file.intersection(label_file)
    
    return len(matching_files)


### Process images function

In [ ]:
def process_images(images_folders, lebel_folders, output_folders): 
    pass
    for i in range(count_matching_filenames(image_folder, image_folder)):
        image_path = os.path.join(image_folder)